[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/Perturbed-Attention-Guidance-jupyter/blob/main/Perturbed_Attention_Guidanc_jupyter.ipynb)

In [ ]:
%cd /content
!pip install -q diffusers accelerate

import numpy as np
import torch
import cv2

from diffusers import StableDiffusionPipeline
from diffusers.utils.torch_utils import randn_tensor

from accelerate.utils import set_seed
from google.colab.patches import cv2_imshow

# load pipeline

pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", custom_pipeline="hyoungwoncho/sd_perturbed_attention_guidance", torch_dtype=torch.float16)

device="cuda"
pipe = pipe.to(device)

# hyperparameters

pag_scale = 5.0
pag_applied_layers_index = ['m0']

seed = 321

# sampling

set_seed(seed)
latent_input = randn_tensor(shape=(1,4,64,64),generator=None, device=device, dtype=torch.float16)

output_baseline = pipe(
    "",
    width=512,
    height=512,
    num_inference_steps=50,
    guidance_scale=0.0,
    pag_scale=0.0,
    pag_applied_layers_index=pag_applied_layers_index,
    latents=latent_input
).images[0]
output_baseline.save(f'uncond_baseline.png')

output_pag = pipe(
    "",
    width=512,
    height=512,
    num_inference_steps=50,
    guidance_scale=0.0,
    pag_scale=pag_scale,
    pag_applied_layers_index=pag_applied_layers_index,
    latents=latent_input
).images[0]
output_pag.save(f'uncond_pag.png')


uncond_baseline = cv2.imread('uncond_baseline.png')
uncond_pag = cv2.imread('uncond_pag.png')
result_image = cv2.hconcat([uncond_baseline, uncond_pag])
cv2_imshow(result_image)